In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os
import cv2
import time
import pandas as pd
from helper import get_penmap

Using TensorFlow backend.


In [2]:
def compute_dist(param1,param2):
    d = 0
    for i in range(0,len(param1)):
        d+=np.nan_to_num(((param1[i]-param2[i])**2)/(param1[i]+param2[i]))
    return d

def dist(param1,param2,n1,n2):
    mean1 = np.mean(param1)
    mean2 = np.mean(param2)
    var1  = np.var(param1)
    var2 = np.var(param2)
     
    d = np.abs(mean1-mean2)
    d/=(var1/n1 + var2/n2)**0.5
    
    return d

def distance_distribution(features):
    same_pens = {}
    diff_pens = {}
    
    for k in range(1,113) :
        for i in ['1','2']: 
            for n in ['1','2']:
                for j in ['1','2','3','4','5','6','7','8'] :
                    for m in ['1','2','3','4','5','6','7','8'] :
                        try:
                            if(i==n and j==m):
                                continue    
                            elif i==n and 'c_'+str(k)+'_'+n+'_'+m+'::'+i+'_'+j not in same_pens:
                                same_pens['c_'+str(k)+'_'+i+'_'+j+'::'+n+'_'+m] = compute_dist(features['c'+str(k)+'_'+i+'_'+j],features['c'+str(k)+'_'+n+'_'+m])
                                #print(' same {}   :{}'.format('c_'+str(k)+'_'+i+'_'+j+'::'+n+'_'+m,same_pens['c_'+str(k)+'_'+i+'_'+j+'::'+n+'_'+m]))
                    
                            elif i!=n and 'c_'+str(k)+'_'+n+'_'+m+'::'+i+'_'+j not in diff_pens :                               
                                diff_pens['c_'+str(k)+'_'+i+'_'+j+'::'+n+'_'+m] = compute_dist(features['c'+str(k)+'_'+i+'_'+j],features['c'+str(k)+'_'+n+'_'+m])
                                #print(' diff {}   :{}'.format('c_'+str(k)+'_'+i+'_'+j+'::'+n+'_'+m,diff_pens['c_'+str(k)+'_'+i+'_'+j+'::'+n+'_'+m]))
                        except:
                            
                            pass
     
    
    same_pens = [ v for v in same_pens.values() ]
    diff_pens = [ v for v in diff_pens.values() ]
   # print('Lengths same pens-{} different pens-{}'.format(len(same_pens),len(diff_pens)))
    return same_pens,diff_pens

def pen_distribution(features,a,pen_out):
    same_pens = {}
    diff_pens = {}
    
    for k in range(1,113) :
        if(k in a[pen_out]):
            for i in ['1','2']: 
                for n in ['1','2']:
                    for j in ['1','2','3','4','5','6','7','8'] :
                        for m in ['1','2','3','4','5','6','7','8'] :
                            try:
                                if(i==n and j==m):
                                    continue    
                                elif i==n and 'c_'+str(k)+'_'+n+'_'+m+'::'+i+'_'+j not in same_pens:
                                    same_pens['c_'+str(k)+'_'+i+'_'+j+'::'+n+'_'+m] =compute_dist(features['c'+str(k)+'_'+i+'_'+j],features['c'+str(k)+'_'+n+'_'+m])
                                    #print(' same {}   :{}'.format('c_'+str(k)+'_'+i+'_'+j+'::'+n+'_'+m,same_pens['c_'+str(k)+'_'+i+'_'+j+'::'+n+'_'+m]))

                                elif i!=n and 'c_'+str(k)+'_'+n+'_'+m+'::'+i+'_'+j not in diff_pens :                               
                                    diff_pens['c_'+str(k)+'_'+i+'_'+j+'::'+n+'_'+m] =compute_dist(features['c'+str(k)+'_'+i+'_'+j],features['c'+str(k)+'_'+n+'_'+m])
                                    #print(' diff {}   :{}'.format('c_'+str(k)+'_'+i+'_'+j+'::'+n+'_'+m,diff_pens['c_'+str(k)+'_'+i+'_'+j+'::'+n+'_'+m]))
                            except:

                                pass

    
    same_pens = [ v for v in same_pens.values() ]
    diff_pens = [ v for v in diff_pens.values() ]
   # print('Lengths same pens-{} different pens-{}'.format(len(same_pens),len(diff_pens)))
    return same_pens,diff_pens



In [3]:
def distance_btw_distributions(feature,a):
    d = []
    same_pens,diff_pens = distance_distribution(feature)
    D = dist(same_pens,diff_pens,1465,1456)
    for j in range(1,15):
        same_pens_i,diff_pens_i= pen_distribution(feature,a,j)
        d1 = len(same_pens_i)/2930*dist(same_pens_i,same_pens,len(same_pens_i),1465)
        d2 = len(diff_pens_i)/2912*dist(diff_pens_i,diff_pens,len(diff_pens_i),1456)
        d.append(d1+d2)
        
    return D-sum(d)

In [4]:
def load_parameters(path,n_param):
    hist_Y = {}
    hist_Cb = {}
    hist_Cr = {}
    a = pd.read_csv(path)
    for k in range(1,113):
        for i in ['1','2']:
            for j in ['1','2','3','4','5','6','7','8']:
                try:
                    
                    hist_Y['c'+str(k)+'_'+i+'_'+j] = a['c'+str(k)+'_'+i+'_'+j][0:n_param] 
                    hist_Cb['c'+str(k)+'_'+i+'_'+j] = a['c'+str(k)+'_'+i+'_'+j][n_param:2*n_param]
                    hist_Cr['c'+str(k)+'_'+i+'_'+j] = a['c'+str(k)+'_'+i+'_'+j][2*n_param:3*n_param]
                    
                    hist_Y['c'+str(k)+'_'+i+'_'+j].reset_index(inplace=True, drop=True)
                    hist_Cb['c'+str(k)+'_'+i+'_'+j].reset_index(inplace=True, drop=True)
                    hist_Cr['c'+str(k)+'_'+i+'_'+j].reset_index(inplace=True, drop=True)
                    
                except:
                    break
    return hist_Y,hist_Cb,hist_Cr

In [5]:
def separate(h1,f):
    hist = {}    
    for k in range(1,113):
        for i in ['1','2']:
            for j in ['1','2','3','4','5','6','7','8']:
                try:
                    hist['c'+str(k)+'_'+i+'_'+j] = np.array([h1['c'+str(k)+'_'+i+'_'+j][f]])
                except:
                    break
    return hist

def concat2(h1,h2):
    hist = {}    
    for k in range(1,113):
        for i in ['1','2']:
            for j in ['1','2','3','4','5','6','7','8']:
                try:
                    hist['c'+str(k)+'_'+i+'_'+j] = np.concatenate([h1['c'+str(k)+'_'+i+'_'+j],h2['c'+str(k)+'_'+i+'_'+j]])
                except:
                    break
    return hist

def concat3(h1,h2,h3):
    hist = {}    
    for k in range(1,113):
        for i in ['1','2']:
            for j in ['1','2','3','4','5','6','7','8']:
                try:
                    hist['c'+str(k)+'_'+i+'_'+j] = np.concatenate([h1['c'+str(k)+'_'+i+'_'+j],h2['c'+str(k)+'_'+i+'_'+j],h3['c'+str(k)+'_'+i+'_'+j]])
                except:
                    break
    return hist

In [6]:
a = get_penmap()

In [7]:
path = '/home/mohit/Desktop/featuresUpdated/statistical_features.csv'
y,cb,cr = load_parameters(path,5)

In [8]:
fe = {}

for i in range(1,6):
    fe[i] = separate(y,i-1)
    fe[i+5] = separate(cb,i-1)
    fe[i+10] = separate(cr,i-1)
    
for i in range(1,16):
    mi = np.min([ v for v in fe[i].values() ])
    mx = np.max([ v for v in fe[i].values() ])
    for key in fe[i].keys():
        fe[i][key]-=mi
        fe[i][key]/=(mx-mi)    


In [9]:
Distances = {}
for i in range(1,16):
    Distances[i] = distance_btw_distributions(fe[i],a)
        
sorted_by_value = sorted(Distances.items(), key=lambda kv: kv[1],reverse =True)
sorted_by_value[0]

(1, 17.330539221563843)

### after comparing all features, we found that fe[1] i.e 1_y is the best and will be the starting point of our forward selection technique.

In [10]:
feycb = {}
feycr  ={}
feycbcr = {}
for i in range(1,6):
    feycb[i] = concat2(fe[i],fe[i+5])
    feycr[i] = concat2(fe[i],fe[i+10])
    feycbcr[i] = concat3(fe[i],fe[i+5],fe[i+10])

In [11]:
# fmax will contain the selected features. dmax will contain the increment by adding the ith feature. 
# Fmax will contain a dict with all selected features concatenated together.

fmax = [1]
dmax = [17.330539221563843]
Fmax = fe[1]
Fmaxycb = feycb[1]
Fmaxycr = feycr[1]
Fmaxycbcr = feycbcr[1]

In [12]:
Dmax = sum(dmax)
f= range(1,6)

In [13]:
while(1):
    Distancesy = {}
    
    for i in set(f)-set(fmax):
        Distancesy[i] = distance_btw_distributions(concat2(Fmax,fe[i]),a) - Dmax
        
    sorted_by_value = sorted(Distancesy.items(), key=lambda kv: kv[1],reverse =True)
    index = sorted_by_value[0][0]
    dd = sorted_by_value[0][1]
    
    Distancesycb = distance_btw_distributions(Fmaxycb,a)-Dmax
        
    Distancesycr = distance_btw_distributions(Fmaxycr,a)-Dmax
    
    print(index,dd,Distancesycb,Distancesycr)
    
    if(dd<Distancesycb or dd<Distancesycr or dd<0):
        if(max(Distancesycb,Distancesycr)>0):
            dmax.append(max(Distancesycb,Distancesycr))
            Dmax = sum(dmax)
        break            
                
    fmax.append(index)
    dmax.append(dd)
    Fmax = concat2(fe[index],Fmax)
    Dmax = sum(dmax)
    Fmaxycb = concat2(feycb[index],Fmaxycb)
    Fmaxycr = concat2(feycr[index],Fmaxycr)
    Fmaxycbcr = concat2(feycbcr[index],Fmaxycbcr)

4 1.9019224424766499 5.676541862680484 3.601562734293463


In [14]:
fmax,dmax

([1], [17.330539221563843, 5.676541862680484])

### after applying our strategy, we found the following selection pattern. 
### 1y -> 1 ycb
### so now we will apply forward selection on cbcr channel with fstart = [1]

In [15]:
while(1):
    Distancesycb = {}
    
    for i in set(f)-set(fmax):
        Distancesycb[i] = distance_btw_distributions(concat2(Fmaxycb,feycb[i]),a)-Dmax
        
    sorted_by_value = sorted(Distancesycb.items(), key=lambda kv: kv[1],reverse =True)
    dd = sorted_by_value[0][1]
    index = sorted_by_value[0][0]
    
    Distancesycbcr = distance_btw_distributions(Fmaxycbcr,a)-Dmax
        
    print(index,dd,Distancesycbcr)
    if(dd<Distancesycbcr or dd<0):
        if(Distancesycbcr>0):
            dmax.append(Distancesycbcr)
            Dmax = sum(dmax)
        break
                
    fmax.append(index)
    dmax.append(dd)
    Dmax = sum(dmax)
    Fmaxycb = concat2(feycb[index],Fmaxycb)
    Fmaxycbcr = concat2(feycbcr[index],Fmaxycbcr)

2 1.77588221350684 3.039936704972046


In [16]:
fmax,dmax

([1], [17.330539221563843, 5.676541862680484, 3.039936704972046])

### sequence of selection

###  1y  -> 1ycb -> 1ycbcr

### so we have to apply forward selection on ycbcr channel with fstart =[1]

In [17]:
while(1):
    Distancesycbcr = {}
    for i in set(f)-set(fmax):
        Distancesycbcr[i] = distance_btw_distributions(concat2(Fmaxycbcr,feycbcr[i]),a)-Dmax
        
    sorted_by_value = sorted(Distancesycbcr.items(), key=lambda kv: kv[1],reverse =True)
    dd = sorted_by_value[0][1]
    index = sorted_by_value[0][0]
    
    print(index,dd)
    if(dd<0):
        break
                
    fmax.append(index)
    dmax.append(dd)
    Dmax = sum(dmax)
    Fmaxycbcr = concat2(feycbcr[index],Fmaxycbcr)

5 -0.13674504005429


In [18]:
fmax,dmax

([1], [17.330539221563843, 5.676541862680484, 3.039936704972046])

# So, f[1] i.e mean is selected in all color channels

In [19]:
#lets save the original features(not normalised) !

best = concat3(separate(y,0),separate(cb,0),separate(cr,0))

In [20]:
best = pd.DataFrame(best)
best

,c100_1_1,c100_1_2,c100_1_3,c100_1_4,c100_2_1,c100_2_2,c100_2_3,c101_1_1,c101_1_2,c101_1_3,...,c99_2_3,c9_1_1,c9_1_2,c9_1_3,c9_1_4,c9_1_5,c9_1_6,c9_1_7,c9_2_1,c9_2_2
0,72.486643,75.916715,68.991495,73.673966,47.518584,45.440278,43.909430,43.503347,42.037456,41.841573,...,44.992625,91.557559,81.518617,80.132029,77.692927,76.910283,79.434095,79.408792,67.531250,70.691566
1,134.569493,134.792074,134.547838,134.412409,128.239646,128.162350,128.126265,129.474020,129.556769,129.348315,...,128.227139,119.373093,118.459441,119.431133,119.322599,120.026528,119.495606,119.349602,115.936298,113.386747
2,126.109175,125.982194,126.091425,125.944343,134.134867,133.838423,133.706979,127.372330,127.478736,127.793820,...,134.044985,172.149792,177.789894,174.333333,175.864290,172.880477,177.645577,176.975078,178.679087,186.748996


In [21]:
best.to_csv('/home/mohit/Desktop/featuresUpdated/statistical_Selectedfeatures.csv')